<a href="https://colab.research.google.com/github/dlskawns/RecSys_and_Retrieval_Study/blob/main/SVD(SingularValueDecomposition).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Surprise 패키지를 활용한 SVD 모델 구현

## SVD 모델의 원리

Singular Vector Decomposition을 이용한 모델.

singular matrix = 역행렬이 없는 매트릭스를 의미
det(A) = 0

```
a = [[a,b],
     [c,d]]
일때,
det(A) = ad - bc

A^-1 = det{A}^-1
```

A: 기존 Input data matrix [m x n] 행렬

U: 왼쪽 singular matrix -> y에 따라 x가 무한이거나 존재하지 않음

$\sum$: r개의 특이값을 갖는 대각행렬

V: 오른쪽 singluar matrix

특이값(Singular Value): A에 대해 $A^TA$한 대칭행렬의 고유값을 구해 루트를 씌워준 값을 의미

고유값(Eigen Value): 임의의 벡터 v에 선형변환 A를 적용했을 때, Av = $\lambda$v를 만족하는 상황에서의 $\lambda$값

고유벡터(Eigen Vector): 위 상황에서의 v값을 의미하며, 고유값을 구한 뒤, 가우스 소거법을 이용해 계산한다. 고유벡터는 영벡터가 아닌 벡터를 의미

SVD 특이값 분해의 이용: 정방행렬이 아닌 m x n 의 행렬 A에 대해서도 대칭행렬을 만들 수 있고 이를 바탕으로 행렬 분해가 가능하며, 차원의 축소 역시도 가능하게 만든다.







In [ ]:
# print(data)

In [ ]:
# model = SVD()

In [ ]:
# cross_validate(model, data[:10], measures=['rmse', 'mae'], cv = 5, verbose = True)

## Numpy package의 SVD를 이용해보기

In [ ]:
import numpy as np
from numpy.linalg import svd

# 4 x 4 행렬 생성
np.random.seed(42)
a = np.random.randn(4,4)  # 평균 0, 표준편차 1의 가우시간 난수 생성
print(np.round(a, 3))

[[ 0.497 -0.138  0.648  1.523]
 [-0.234 -0.234  1.579  0.767]
 [-0.469  0.543 -0.463 -0.466]
 [ 0.242 -1.913 -1.725 -0.562]]


### $U$, $\sum$, $V^T$ 확인해보기

In [ ]:
# SVD 함수 적용해서 각 U, Sigma, Vt 확인해보기
U, Sigma, Vt = svd(a)
print('U:', np.round(U,3))
print('Sigma:', np.round(Sigma,3))  # 시그마의 경우, 대각행렬의 요소값인 Singular Value를 내림차순으로 가져온다.
print('Vt:', np.round(Vt,3))

U: [[-0.373 -0.598  0.642 -0.302]
 [-0.488 -0.35  -0.745 -0.289]
 [ 0.113  0.444  0.062 -0.887]
 [ 0.781 -0.568 -0.168 -0.197]]
Sigma: [3.08  1.926 0.92  0.342]
Vt: [[ 0.021 -0.412 -0.783 -0.466]
 [-0.291  0.775 -0.086 -0.554]
 [ 0.461  0.479 -0.544  0.512]
 [ 0.838  0.017  0.289 -0.462]]


In [ ]:
# 시그마의 대각행렬을 원상복구 해주는 방법: np.diag
Sigma_matrix = np.diag(Sigma)
print('Sigma :', np.round(Sigma_matrix, 3))

Sigma : [[3.08  0.    0.    0.   ]
 [0.    1.926 0.    0.   ]
 [0.    0.    0.92  0.   ]
 [0.    0.    0.    0.342]]


### A = $U \sum V^T$ 만들어보기

In [ ]:
a_ = np.dot(np.dot(U,Sigma_matrix), Vt)
print(np.round(a, 3) )  # 맨 처음의 매트릭스 a와 같음을 확인할 수 있다.

[[ 0.497 -0.138  0.648  1.523]
 [-0.234 -0.234  1.579  0.767]
 [-0.469  0.543 -0.463 -0.466]
 [ 0.242 -1.913 -1.725 -0.562]]


### COMPACT SVD 구현해보기


In [ ]:
# 데이터 의존도가 서로 존재하는 행렬을 만들어본다.
a[2] = a[0]+a[1]
a[3] = a[0]

In [ ]:
print(np.round(a,3))

[[ 0.497 -0.138  0.648  1.523]
 [-0.234 -0.234  1.579  0.767]
 [ 0.263 -0.372  2.227  2.29 ]
 [ 0.497 -0.138  0.648  1.523]]


In [ ]:
# SVD 분할진행
U, Sigma, Vt = svd(a)
print('U:', np.round(U,3))
print('Sigma:', np.round(Sigma,3))  # 시그마의 3,4번째 특잇값이 0임을 확인할 수 있다.
print('Vt:', np.round(Vt,3))

U: [[-0.381  0.505  0.775  0.003]
 [-0.374 -0.679  0.256  0.579]
 [-0.755 -0.174 -0.256 -0.579]
 [-0.381  0.505 -0.519  0.575]]
Sigma: [4.266 1.19  0.    0.   ]
Vt: [[-0.115  0.111 -0.648 -0.745]
 [ 0.517  0.071 -0.677  0.52 ]
 [-0.848 -0.    -0.328  0.416]
 [ 0.024 -0.991 -0.121 -0.047]]


In [ ]:
# 특이값 행렬 시그마의 0을 제거한 Compact SVD를 진행한다.
U_ = U[:, :2]
Sigma_ = np.diag(Sigma[:2])
Vt_ = Vt[:2]
print('U_:', np.round(U_,3))
print('Sigma_:', np.round(Sigma_,3))  
print('Vt_:', np.round(Vt_,3))

U_: [[-0.381  0.505]
 [-0.374 -0.679]
 [-0.755 -0.174]
 [-0.381  0.505]]
Sigma_: [[4.266 0.   ]
 [0.    1.19 ]]
Vt_: [[-0.115  0.111 -0.648 -0.745]
 [ 0.517  0.071 -0.677  0.52 ]]


In [ ]:
# 다시 복구하기

a_ = np.dot(np.dot(U_, Sigma_), Vt_)
print(np.round(a_, 3))

[[ 0.497 -0.138  0.648  1.523]
 [-0.234 -0.234  1.579  0.767]
 [ 0.263 -0.372  2.227  2.29 ]
 [ 0.497 -0.138  0.648  1.523]]


## Scipy 패키지를 이용해 SVD 구현하기

full_matrices = True -> U를 (M,M), Vt를 (N,N)으로 분할
full_matrices = False -> U를 (M,k), Vt를 (k,N)으로 분할



In [ ]:
from scipy.sparse.linalg import svds
from scipy.linalg import svd

# 원본 행렬 생성
np.random.seed(12)
matrix = np.random.random((6,6))
print( np.round(matrix, 3))

[[0.154 0.74  0.263 0.534 0.015 0.919]
 [0.901 0.033 0.957 0.137 0.284 0.606]
 [0.944 0.853 0.002 0.521 0.552 0.485]
 [0.768 0.161 0.765 0.021 0.135 0.116]
 [0.31  0.671 0.471 0.816 0.29  0.733]
 [0.703 0.328 0.335 0.978 0.625 0.95 ]]


In [ ]:
U, Sigma, Vt = svd(matrix, full_matrices = False)
print('U:', np.round(U,3))
print('Sigma:', np.round(Sigma,3)) 
print('Vt:', np.round(Vt,3))

U: [[-0.363  0.377  0.435  0.501 -0.449  0.292]
 [-0.385 -0.64   0.235 -0.061 -0.383 -0.487]
 [-0.443  0.149 -0.818  0.259 -0.13  -0.17 ]
 [-0.25  -0.582 -0.058  0.308  0.386  0.593]
 [-0.432  0.254  0.29   0.107  0.695 -0.413]
 [-0.524  0.151  0.006 -0.756 -0.055  0.357]]
Sigma: [3.145 1.289 0.799 0.603 0.301 0.058]
Vt: [[-0.482 -0.369 -0.329 -0.429 -0.269 -0.517]
 [-0.497  0.396 -0.611  0.414 -0.004  0.227]
 [-0.555 -0.226  0.54   0.099 -0.373  0.446]
 [ 0.008  0.768  0.177 -0.418 -0.442 -0.092]
 [-0.212  0.181  0.397  0.535  0.106 -0.684]
 [ 0.41  -0.182 -0.193  0.417 -0.762 -0.079]]


### Scipy svd를 통해 Truncated SVD 구현해보기

In [ ]:
#사용할 특이값 개수 지정
num_comp = 4

U_, Sigma_, Vt_ = svds(matrix, k = num_comp)
print('U_:', np.round(U_,3))          # 4개의 특이값에 맞춰 행렬곱을 취할 수 있도록 행렬이 분할되었다.
print('Sigma_:', np.round(Sigma_,3))  # 시그마 행렬에서는 4개의 특이값만을 가진 대각행렬이 된 것을 볼 수 있다.
print('Vt_:', np.round(Vt_,3))        # U와 마찬가지.

U_: [[ 0.501  0.435  0.377  0.363]
 [-0.061  0.235 -0.64   0.385]
 [ 0.259 -0.818  0.149  0.443]
 [ 0.308 -0.058 -0.582  0.25 ]
 [ 0.107  0.29   0.254  0.432]
 [-0.756  0.006  0.151  0.524]]
Sigma_: [0.603 0.799 1.289 3.145]
Vt_: [[ 0.008  0.768  0.177 -0.418 -0.442 -0.092]
 [-0.555 -0.226  0.54   0.099 -0.373  0.446]
 [-0.497  0.396 -0.611  0.414 -0.004  0.227]
 [ 0.482  0.369  0.329  0.429  0.269  0.517]]


In [ ]:
# 행렬 다시 합치기 -> Truncated 이므로 0이 아닌 Singular Value도 없는 상태에서의 행렬곱을 취한것이므로, 그 값이 완전히 맨 처음의 값과 일치하진 않는다.

a_ = np.dot(np.dot(U_, np.diag(Sigma_)), Vt_)
print(np.round(a_, 3))

[[ 0.119  0.768  0.32   0.599  0.042  0.828]
 [ 0.888  0.049  0.997  0.211  0.274  0.525]
 [ 0.94   0.858  0.016  0.546  0.549  0.458]
 [ 0.779  0.146  0.725 -0.056  0.149  0.198]
 [ 0.364  0.629  0.383  0.714  0.249  0.874]
 [ 0.691  0.334  0.345  0.978  0.642  0.941]]


## Scikit-Learn패키지를 이용해 SVD 해보기

In [ ]:
# 원본 행렬 생성
np.random.seed(12)
matrix = np.random.random((6,6))
print( np.round(matrix, 3))

[[0.154 0.74  0.263 0.534 0.015 0.919]
 [0.901 0.033 0.957 0.137 0.284 0.606]
 [0.944 0.853 0.002 0.521 0.552 0.485]
 [0.768 0.161 0.765 0.021 0.135 0.116]
 [0.31  0.671 0.471 0.816 0.29  0.733]
 [0.703 0.328 0.335 0.978 0.625 0.95 ]]


In [ ]:
from sklearn.decomposition import TruncatedSVD

tsvd = TruncatedSVD(n_components=4)
tsvd.fit(matrix)
a_tr = tsvd.transform(matrix)
a_tr

array([[ 1.14188038, -0.48539404, -0.34749425, -0.30191158],
       [ 1.20960458,  0.82529355, -0.1876061 ,  0.03686302],
       [ 1.39327376, -0.19179119,  0.65318726, -0.15636906],
       [ 0.78638259,  0.75033534,  0.04616277, -0.18548138],
       [ 1.35902108, -0.32771741, -0.23137577, -0.06471603],
       [ 1.64798258, -0.1950719 , -0.00497557,  0.4562135 ]])

## Matrix 유사도를 통한 추천

각 영화를 본 내용을 바탕으로 가장 유사한 유저를 찾아 해당 유저가 본 영화를 추천한다.



### 인접행렬 및 유사도를 통한 가장 비슷한 유저 찾기

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 4.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630194 sha256=87b62e2f5b059d8afc80f7f3786e4d7bfe7f2fa3f8d5db143868d1c76a9bb3b0
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:

data = Dataset.load_builtin('ml-100k', prompt = False)
raw_data = np.array(data.raw_ratings, dtype = int)

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
# user와 movie의 index를 0부터 시작할 수 있도록 1씩 빼준다.
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
# shape 자체는 위에서 1씩 빼준것을 감안하여 1씩 더해준 것을 shape으로 둔다.
shape= (n_users +1, n_movies +1)
shape

(943, 1682)

In [ ]:
# 인접행렬 생성
adj_matrix = np.ndarray(shape, dtype=int)
adj_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
adj_matrix.shape  # 943개의 샘플(유저)과 1682개의 features(영화)에 대한 데이터

(943, 1682)

In [ ]:
for  user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = 1.  # 데이터가 있는 경우, 1로 설정해준다.
adj_matrix


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [ ]:
len(adj_matrix)

943

In [ ]:
len(adj_matrix[0])

1682

In [ ]:
# 추천 진행할 id와 matrix에서의 해당 id 설정
id, vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
  # 유저 id와 선택 id가 다른경우, 유사도를 체크 한다. 선택 id에 대해서는 할 필요가 없기 때문에 생랼
  if id != user_id:
    similarity = np.dot(vector, user_vector)
    if similarity > best_match:  # 유사도가 기존 best_match보다 높을 경우
      best_match = similarity # best_match에 해당 유사도 값을 넣어준다.
      best_match_id = user_id # 현재 유사도를 계산한 유저 id를 best match id로 선정
      best_match_vector = user_vector # 현재 유사도를 계산한 vector를 넣는다.

print(f'Best Match (유사도 스코어): {best_match},\nBest Match ID (id "{id}"번과 가장 유사한 id): {best_match_id}')


Best Match (유사도 스코어): 183,
Best Match ID (id "0"번과 가장 유사한 id): 275


### 특정 ID('0')에 대한 추천 리스트를 뽑아보기

In [ ]:
best_match_vector

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
vector

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
max(vector)

1

In [ ]:
recommendation = []
for i, log in enumerate(zip(vector, best_match_vector)):
  log1, log2 = log # 유저 0이 본 영화정보 vector(log1)와 유저 0과 비슷한 유저가 본 영화정보 best_match_vector(log2)를 비교하기 위함
  # 기존에 본 영화(log1)은 배제하고(0인 경우), 보지 않았던 영화 중, 가장 유사한 유저가 본 영화(log2가 1인경우) 
  if log1 < 1. and log2 > 0.:
    recommendation.append(i)
print(recommendation)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

### 유클리드 거리를 이용한 추천

$euclidean = \sqrt\displaystyle\sum_{d=i}^{D} (A_i - B_i)^2$

유클리드 거리가 가까울 수록 지정 id와 유사한 사용자로 취급

In [ ]:
for user_id, user_vector in enumerate(adj_matrix):
  print(user_vector)

[1 1 1 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 1 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 1 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 1 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[1 1 0 ... 0 0 0]
[1 0 1 ... 0 0 0]
[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 ...

In [ ]:
id, vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector

print(f'Best Match: {best_match} \nBest Match ID: {best_match_id} \nBest_Match_Vector: {best_match_vector}')

Best Match: 14.832396974191326 
Best Match ID: 737 
Best_Match_Vector: [1 1 0 ... 0 0 0]


In [ ]:
recommendation = []
for i, log in enumerate(zip(vector, best_match_vector)):
  log1, log2 = log
  if log1 <1. and log2 >0.:
    recommendation.append(i)
print(recommendation)


[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


### 코사인 유사도를 이용한 추천

두 벡터가 이루는 값을 계산(내적)해 추천  

### $cos\theta = \frac{A * B}{||A||*||B||}$

In [ ]:
def compute_cos_similarity(v1,v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1, v2)
  return dot / (norm1 * norm2)


In [ ]:
id, vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if id != user_id:
    cos_similarity = compute_cos_similarity(vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print(f'Best Match: {best_match} \nBest Match ID: {best_match_id} \nBest_Match_Vector: {best_match_vector}')

Best Match: 0.5278586163659506 
Best Match ID: 915 
Best_Match_Vector: [1 1 1 ... 0 0 1]


In [ ]:
recommendation = []
for i, log in enumerate(zip(vector, best_match_vector)):
  log1, log2 = log
  if log1 <1. and log2 >0.:
    recommendation.append(i)
print(recommendation)


[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


### 영화 점수를 살려서 진행해보기
결과가 똑같음

In [ ]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
id, vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if id != user_id:
    cos_similarity = compute_cos_similarity(vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print(f'Best Match: {best_match} \nBest Match ID: {best_match_id} \nBest_Match_Vector: {best_match_vector}')

Best Match: 0.569065731527988 
Best Match ID: 915 
Best_Match_Vector: [4 3 3 ... 0 0 3]


## sklearn 패키지 SVD를 이용해 추천하기

User Based Collaborative Filtering

In [ ]:
from sklearn.decomposition import randomized_svd

# SVD를 이용해 특이값 2개짜리의 행렬로 분해한 뒤 곱해준다.
U, S, V = randomized_svd(adj_matrix, n_components = 2 )
S = np.diag(S)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


In [ ]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

np.matmul과 np.dot의 차이 확인하기:
https://jlog1016.tistory.com/80

보통 n차원 행렬인 경우 np.matmul을 이용

In [ ]:
matmuled_matrix = np.matmul(np.matmul(U,S),V)

In [ ]:
# 행렬분해한 U 행렬을 사용 - 유저기반 콜라보러티브 필터링
id, vector = 0, U[0]
best_match_id, best_match_vector, best_match = -1, [], -1

for user_id, user_vector in enumerate(U):
  if id != user_id:
    cosine_similarity = compute_cos_similarity(vector, user_vector)
    if cosine_similarity > best_match:
      best_match_id = user_id
      best_match_vector = user_vector
      best_match = cosine_similarity

print(f'Best Match: {best_match} \nBest Match ID: {best_match_id} \nBest_Match_Vector: {best_match_vector}')

Best Match: 0.9999942295956324 
Best Match ID: 235 
Best_Match_Vector: [0.03467744 0.00326754]


In [ ]:
matmuled_matrix[best_match_id]

array([2.06180836e+00, 7.75911609e-01, 4.19834251e-01, ...,
       3.16276312e-04, 7.43876997e-03, 7.20158124e-03])

In [ ]:
recommendation = []
for i, log in enumerate(zip(adj_matrix[id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 <1. and log2 >0.:
    recommendation.append(i)
print(recommendation)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


In [ ]:
# 행렬분해한 VT 행렬을 사용 - 아이템기반 콜라보러티브 필터링
id, vector = 0, V.T[0]
best_match_id, best_match_vector, best_match = -1, [], -1

for user_id, user_vector in enumerate(U):
  if id != user_id:
    cosine_similarity = compute_cos_similarity(vector, user_vector)
    if cosine_similarity > best_match:
      best_match_id = user_id
      best_match_vector = user_vector
      best_match = cosine_similarity

print(f'Best Match: {best_match} \nBest Match ID: {best_match_id} \nBest_Match_Vector: {best_match_vector}')

Best Match: 0.9999999758529564 
Best Match ID: 873 
Best_Match_Vector: [ 0.00932328 -0.00848059]


# 8/28 SVD를 통한 MovieLens 모델링

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd

# 데이터 불러오기
ratings = pd.read_table('/content/drive/MyDrive/ml-1m/ratings.dat', sep="::", header = None)
ratings = ratings.rename(columns = {0: 'userId',1:'movieId',2:'rating', 3:'time'})
print(ratings.shape)
ratings.head(5)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(1000209, 4)


,userId,movieId,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 유저, 영화의 평점 행렬로 변환
adj_matrix_rating = ratings.pivot(index = 'userId', columns = 'movieId', values = 'rating')

In [4]:
# (유저수, 영화 수) 파악
print(adj_matrix_rating.shape)

# 유저 - 영화 평점행렬 일부 보기
adj_matrix_rating.head(5)

(6040, 3706)


movieId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import copy

adj_matrix = adj_matrix_rating.copy()
# 행렬의 결측치(사용자가 보지 않은 영화)에 대해 해당 사용자의 평균 평점으로 대치해줌
for i in range(adj_matrix_rating.shape[0]):
  adj_matrix.iloc[i] = adj_matrix_rating.iloc[i].fillna(adj_matrix_rating.iloc[i].mean())
adj_matrix.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userId,,,,,,,,,,,,,,,,,,,,,
1,5.000000,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,...,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679
2,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,...,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178
3,3.901961,3.901961,3.901961,3.901961,3.901961,3.901961,3.901961,3.901961,3.901961,3.901961,...,3.901961,3.901961,3.901961,3.901961,3.901961,3.901961,3.901961,3.901961,3.901961,3.901961
4,4.190476,4.190476,4.190476,4.190476,4.190476,4.190476,4.190476,4.190476,4.190476,4.190476,...,4.190476,4.190476,4.190476,4.190476,4.190476,4.190476,4.190476,4.190476,4.190476,4.190476
5,3.146465,3.146465,3.146465,3.146465,3.146465,2.000000,3.146465,3.146465,3.146465,3.146465,...,3.146465,3.146465,3.146465,3.146465,3.146465,3.146465,3.146465,3.146465,3.146465,3.146465


In [6]:
import numpy as np
from numpy.linalg import svd
#numpy를 통한 svd 진행
U, Sigma, Vt = svd(adj_matrix)

In [7]:
Sigma.shape

(3706,)

In [8]:
print(f'특잇값 대각행렬 개수: \n{Sigma.shape[0]}개')
print(f'특잇값 상위 30개: \n{Sigma[:30]}')
print(f'특잇값 하위 30개: \n{Sigma[-30:]}')

특잇값 대각행렬 개수: 
3706개
특잇값 상위 30개: 
[17636.38786959   218.37248025   130.44115489   100.3386695
    93.44339761    87.43051401    83.7291781     77.32125892
    73.65106722    69.76130371    68.20570232    66.86023041
    64.54553212    62.82188713    62.11785242    60.64357302
    59.51223716    58.95154867    58.04919338    56.88837046
    56.46499374    55.52009236    54.82594478    54.17839014
    54.0343861     53.35627703    53.03685309    52.73950587
    52.21476985    51.95324078]
특잇값 하위 30개: 
[1.74553768e-12 1.74553768e-12 1.74553768e-12 1.74553768e-12
 1.74553768e-12 1.74553768e-12 1.74553768e-12 1.74553768e-12
 1.74553768e-12 1.74553768e-12 1.74553768e-12 1.74553768e-12
 1.74553768e-12 1.74553768e-12 1.74553768e-12 1.74553768e-12
 1.74553768e-12 1.74553768e-12 1.74553768e-12 1.74553768e-12
 1.74553768e-12 1.74553768e-12 1.74553768e-12 1.74553768e-12
 1.74553768e-12 1.74553768e-12 1.74553768e-12 1.74553768e-12
 1.74553768e-12 1.74553768e-12]


In [9]:
# singular value 개수 초기화
cnt, s = 0, 0

# 대각행렬 singular value 개수 만큼 반복
for i in range(len(Sigma)):
  cnt += 1
  s += Sigma[i]
  
  # 전체 singular value의 90%까지 파악할 수 있는 개수가 되면 반복 중지
  if s >= sum(Sigma)*0.9:
    break
print(f'90%의 singular value 개수: \n{cnt}개')

90%의 singular value 개수: 
1757개


In [10]:
# 최적의 singular value 개수 만큼 차원을 축소하여 행렬 곱을 취해 값을 추정
U = U[:, :1757]
Sigma = np.diag(Sigma[:1757])
Vt = Vt[:1757]

adj = np.dot(np.dot(U, Sigma), Vt)

In [11]:
adj

array([[4.99567318, 4.18417638, 4.18703922, ..., 4.18597718, 4.18240146,
        4.18495941],
       [3.702871  , 3.69690202, 3.70878063, ..., 3.7142205 , 3.75326943,
        3.72763303],
       [3.89644532, 3.8807421 , 3.91087832, ..., 3.9287229 , 3.88535235,
        3.88597234],
       ...,
       [3.80416717, 3.80088489, 3.79608201, ..., 3.8057551 , 3.79277019,
        3.82581774],
       [3.86142863, 3.85604983, 3.89643772, ..., 3.90745948, 3.91366979,
        3.86017462],
       [2.98894393, 3.57315194, 3.57603707, ..., 3.57100089, 3.62275799,
        3.60490456]])

### 평가하기

오프라인 평가를 하기 위해선 train - test 셋으로 나눈뒤 test 셋에 내용이 train 셋에 있는 경우만 그 선호도 관련 오차를 계산해본다. 



In [22]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ratings, test_size = 0.3, random_state=2)


In [23]:
U, Sigma, Vt = svd(adj_matrix)

In [24]:
import tqdm
def svd_method(df, k = 30):
  """
  무비렌즈 한정 svd 진행. 
  유저별 영화에 대한 평점 테이블을 pivot 진행하여 svd 후 특정 k차원으로 축소하여 행렬곱 진행
  k값 default: 30
  """
  df_pivot = df.pivot(index = 'userId', columns = 'movieId', values = 'rating')
  for i in range(df_pivot.shape[0]):
    df_pivot.iloc[i] = df_pivot.iloc[i].fillna(df_pivot.iloc[i].mean())
  U, Sigma, Vt = np.linalg.svd(df_pivot)
  U = U[:, :k]
  Sigma = np.diag(Sigma[:k]) 
  Vt = Vt[:k]
  svd_df = pd.DataFrame(np.dot(np.dot(U, Sigma), Vt), columns = df_pivot.columns)
  return svd_df


In [25]:
result_train = svd_method(train)

In [26]:
result_train

movieId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.406719,4.237895,4.232751,4.227334,4.225449,4.232631,4.236012,4.222471,4.231395,4.205404,...,4.219884,4.228557,4.227502,4.233560,4.230812,4.234241,4.226189,4.231685,4.233957,4.225701
1,3.773742,3.536092,3.635949,3.588707,3.621065,3.613953,3.614124,3.619457,3.624868,3.567092,...,3.618290,3.635277,3.591854,3.607248,3.635388,3.625367,3.645536,3.638220,3.636207,3.643587
2,3.933157,3.853219,3.903122,3.905765,3.909692,3.907759,3.929189,3.908893,3.898288,3.917921,...,3.908084,3.905554,3.903522,3.922512,3.903915,3.852818,3.887725,3.905684,3.909697,3.896438
3,4.221256,4.238438,4.261530,4.238799,4.258671,4.239076,4.235123,4.236975,4.224855,4.225954,...,4.223116,4.233445,4.220218,4.216499,4.228287,4.227236,4.237920,4.230017,4.230131,4.219167
4,3.530989,3.229731,3.205670,3.261263,3.186778,3.204552,3.222422,3.223244,3.229376,3.206572,...,3.221656,3.209720,3.211343,3.258488,3.211510,3.205123,3.284182,3.215597,3.227068,3.219447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,3.907816,3.060213,3.487437,3.340842,3.410343,3.152613,3.314705,3.298656,3.237734,3.055208,...,3.227762,3.289536,3.128244,3.108018,3.337385,3.337974,3.645747,3.322584,3.292341,3.347335
6036,3.829205,3.725653,3.779356,3.751018,3.766757,3.717269,3.771798,3.777958,3.814557,3.790586,...,3.773335,3.800333,3.724321,3.776933,3.782133,3.775558,3.791170,3.803606,3.800325,3.785016
6037,4.108660,4.068180,4.056101,4.059848,4.068521,4.064084,4.058407,4.064774,4.071208,4.070060,...,4.064332,4.059684,4.069592,4.058485,4.062609,4.043853,4.058960,4.057798,4.062946,4.064576
6038,3.949719,3.846331,3.870498,3.873213,3.870104,3.849711,3.865330,3.880304,3.883132,3.886553,...,3.887927,3.878939,3.895716,3.886013,3.885571,3.850680,3.871746,3.881901,3.883975,3.887669


In [27]:
def eval(train_result,train, test):
  """
  train_result: k 값에 따라 축소 및 재결합된 행렬
  train: train 데이터
  test: test 데이터
  """
  result = []
  for _, row in test.iterrows():
    # 예측해야 하는 유저, 아이템, 평점
    user = row['userId']
    item = row['movieId']
    true_rating = row['rating']
    # print(user, item, true_rating)
    if user in train_result.index.values and item in train_result.columns.values:
      pred = train_result.loc[user][item]
    else:
      # 없으면 유저 평점의
      if user in train['userId'].values:
        pred = train[train['userId']==user]['rating'].mean()
      else:
        pred = test[test['userId']==user]['rating'].mean()
    result.append([user, item, true_rating, pred])
  df_result = pd.DataFrame(result, columns = ['userId','itemId','rating','prediction'])
  # print(f"RMSE: {np.sqrt(np.square(df_result['prediction']-df_result['rating']).mean())}")
  return np.sqrt(np.square(df_result['prediction']-df_result['rating']).mean())
    
    

In [28]:
eval(result_train, train, test)

1.2030242444281536

### 오차가 적은 k(파라미터)를 찾기


In [36]:
def training(train, test, k_list):
  """
  train: train 데이터
  test: test 데이터
  k_list: 체크하고자 하는 k 리스트
  """
  result_dict = {}
  for k_num in k_list:
    r_train = svd_method(train, k_num)
    rmse = eval(r_train, train, test)
    result_dict.update({k_num:rmse})
  result_sorted = sorted(result_dict.items(), key=lambda item: item[1], reverse=False)
  print(f'rmse 기준 오차가 가장 적은 k와 rmse는 {result_sorted[0]}입니다.')
  print('이외 결과는 아래와 같습니다.')
  for i in result_sorted:
    print(i)
  
    
    

In [37]:
training(train, test, [5, 10, 15, 20, 25, 30])

rmse 기준 오차가 가장 적은 k와 rmse는 (5, 1.198314807092456)입니다.
이외 결과는 아래와 같습니다.
(5, 1.198314807092456)
(10, 1.1994050462231614)
(15, 1.2002991723839531)
(20, 1.2014370640023522)
(25, 1.2023138112376042)
(30, 1.2030242444281536)


### 추천하기


In [46]:
def recommend(original_data, svd_result, user_number, n=10):
  """
  유저와 추천 개수를 선택하면 해당 개수만큼 출력해주는 함수
  original_data: 결측값이 존재하는 상태의 pivot 행렬
  svd_result: svd 결과 행렬
  user_number: 유저 식별 번호
  n: 추천할 아이템 개수
  """
  
  rec_u_i = {}
  for i in original_data.index:
    rec_u_i.update({i: list(original_data.iloc[i-1].dropna().index)})
  # 이전에 봤던 영화는 삭제
  user_df = svd_result.drop(columns = rec_u_i[user_number])
  # 정렬을 통해 선호도가 높은 순으로 10개 출력
  return user_df.iloc[user_n].sort_values(ascending = False)[:10].index
  

In [41]:
adj_matrix_rating.head(2)

movieId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
adj_matrix.head(2)

movieId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userId,,,,,,,,,,,,,,,,,,,,,
1,5.000000,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,...,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679,4.188679
2,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,...,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178,3.713178


In [48]:
# 300번 유저의 영화추천
recommend(adj_matrix_rating, adj_matrix, 300)

Int64Index([3210, 785, 1148, 441, 356, 1961, 1962, 2728, 1449, 2795], dtype='int64', name='movieId')